In [6]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from docx import Document

import datetime
import time
import pandas as pd
import re

In [2]:
# 设置 EdgeOptions
options = EdgeOptions()
options.add_argument("--headless")  # 无头模式
# 初始化 WebDriver
driver = webdriver.Edge(service=EdgeService(EdgeChromiumDriverManager().install()), options=options)

today = datetime.datetime.now().strftime("%m-%d")
yesterday = (datetime.datetime.now() - datetime.timedelta(days=1)).strftime("%m-%d")
companies = pd.read_csv("data/sichuan_new.csv")

In [ ]:
# 访问页面
for index, row in companies.iterrows():
    # 打开 dataweb 的网址
    driver.get(row["dataweb"])
    time.sleep(2)  # 等待页面加载

    # 使用 BeautifulSoup 解析页面内容
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # 找到目标 <div id="gsgk" class="module">
    gsgk_div = soup.find("div", id="gsgk", class_="module")
    if gsgk_div:
        # 找到 <div class="tab-more"> 并提取 <a> 标签中的 href 属性
        titlebar = gsgk_div.find("div", class_="titlebar")
        if titlebar:
            link = titlebar.find("a")
            if link and "href" in link.attrs:
                companies.at[index, "company_link"] = link["href"]  # 存储到新列
                print(link["href"])
            else:
                companies.at[index, "company_link"] = None
        else:
            companies.at[index, "company_link"] = None
    else:
        companies.at[index, "company_link"] = None

# 关闭 WebDriver
driver.quit()

https://emweb.securities.eastmoney.com/pc_hsf10/pages/index.html?type=web&code=SZ002466&color=b#/gsgk
https://emweb.securities.eastmoney.com/pc_hsf10/pages/index.html?type=web&code=SZ000598&color=b#/gsgk
https://emweb.securities.eastmoney.com/pc_hsf10/pages/index.html?type=web&code=SH601838&color=b#/gsgk
https://emweb.securities.eastmoney.com/pc_hsf10/pages/index.html?type=web&code=SZ002312&color=b#/gsgk
https://emweb.securities.eastmoney.com/pc_hsf10/pages/index.html?type=web&code=SZ300841&color=b#/gsgk
https://emweb.securities.eastmoney.com/pc_hsf10/pages/index.html?type=web&code=SH600603&color=b#/gsgk
https://emweb.securities.eastmoney.com/pc_hsf10/pages/index.html?type=web&code=SZ000876&color=b#/gsgk
https://emweb.securities.eastmoney.com/pc_hsf10/pages/index.html?type=web&code=SH601208&color=b#/gsgk
https://emweb.securities.eastmoney.com/pc_hsf10/pages/index.html?type=web&code=SZ000568&color=b#/gsgk
https://emweb.securities.eastmoney.com/pc_hsf10/pages/index.html?type=web&code=SZ0

In [5]:
companies.to_csv("data/sichuan_new.csv", index=False)

In [2]:
# 设置 EdgeOptions
options = EdgeOptions()
options.add_argument("--headless")  # 无头模式
# 初始化 WebDriver
driver = webdriver.Edge(service=EdgeService(EdgeChromiumDriverManager().install()), options=options)

today = datetime.datetime.now().strftime("%m-%d")
yesterday = (datetime.datetime.now() - datetime.timedelta(days=1)).strftime("%m-%d")
companies = pd.read_csv("data/sichuan_new.csv")

In [3]:
def extract_table_data(table_div_class):
    """ 从指定的div下的table提取所有th-td结构的数据，支持一个tr下多个th-td组合 """
    data = {}
    table_div = soup.find("div", class_=table_div_class)
    if table_div:
        table = table_div.find("table")
        if table:
            for row in table.find_all("tr"):
                th_list = row.find_all("th")
                td_list = row.find_all("td")
                for th, td in zip(th_list, td_list):  # 处理多个th-td的情况
                    key = th.get_text(strip=True)
                    value = td.get_text(strip=True)
                    if key in data:
                        if isinstance(data[key], list):  # 如果已有多个值，追加到列表
                            data[key].append(value)
                        else:
                            data[key] = [data[key], value]  # 变成列表存储多个值
                    else:
                        data[key] = value
    return data

In [4]:
def save_to_word(company_name, company_info, issuance_info):
    """ 将公司信息和发行相关信息保存到 Word 文件 """
    doc = Document()

    # 添加公司名称作为标题
    doc.add_heading(company_name, level=1)

    # 添加公司基本资料
    doc.add_heading("公司基本资料", level=2)
    for key, value in company_info.items():
        if isinstance(value, list):
            value = ", ".join(value)  # 如果值是列表，转换为逗号分隔的字符串
        doc.add_paragraph(f"{key}: {value}")

    # 添加发行相关信息
    doc.add_heading("发行相关信息", level=2)
    for key, value in issuance_info.items():
        if isinstance(value, list):
            value = ", ".join(value)
        doc.add_paragraph(f"{key}: {value}")

    # 保存 Word 文件
    filename = f"infobase/{company_name}基本信息.docx"
    doc.save(filename)
    print(f"Word 文件已保存: {filename}")

In [8]:
def clean_company_name(name):
    """ 清理公司名称，去除'...'、'*'等特殊字符，仅保留文字 """
    return re.sub(r"[.*]", "", name).strip()

In [9]:
# 遍历每个公司的 company_link
for _, row in companies.iterrows():
    company_name = row["name"]
    clean_name = clean_company_name(company_name)
    link = row["company_link"]
    
    # 打开链接
    driver.get(link)
    time.sleep(3)  # 等待页面加载
    
    # 使用 BeautifulSoup 解析 HTML
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    # 提取基本信息
    company_info = extract_table_data("jbzl_table")
    issuance_info = extract_table_data("fxxg_table")
    save_to_word(clean_name, company_info, issuance_info)

Word 文件已保存: infobase/天齐锂业基本信息.docx
Word 文件已保存: infobase/兴蓉环境基本信息.docx
Word 文件已保存: infobase/成都银行基本信息.docx
Word 文件已保存: infobase/川发龙蟒基本信息.docx
Word 文件已保存: infobase/康华生物基本信息.docx
Word 文件已保存: infobase/广汇物流基本信息.docx
Word 文件已保存: infobase/新 希 望基本信息.docx
Word 文件已保存: infobase/东材科技基本信息.docx
Word 文件已保存: infobase/泸州老窖基本信息.docx
Word 文件已保存: infobase/川能动力基本信息.docx
Word 文件已保存: infobase/四川路桥基本信息.docx
Word 文件已保存: infobase/五 粮 液基本信息.docx
Word 文件已保存: infobase/红旗连锁基本信息.docx
Word 文件已保存: infobase/成都华微基本信息.docx
Word 文件已保存: infobase/国城矿业基本信息.docx
Word 文件已保存: infobase/雅化集团基本信息.docx
Word 文件已保存: infobase/雷电微力基本信息.docx
Word 文件已保存: infobase/宜宾纸业基本信息.docx
Word 文件已保存: infobase/海创药业基本信息.docx
Word 文件已保存: infobase/坤恒顺维基本信息.docx
Word 文件已保存: infobase/中建环能基本信息.docx
Word 文件已保存: infobase/巨星农牧基本信息.docx
Word 文件已保存: infobase/成都先导基本信息.docx
Word 文件已保存: infobase/航发科技基本信息.docx
Word 文件已保存: infobase/云图控股基本信息.docx
Word 文件已保存: infobase/国光股份基本信息.docx
Word 文件已保存: infobase/成都燃气基本信息.docx
Word 文件已保存: infobase/德龙汇能基本信息.docx
Word 文件已保存: infoba

In [ ]:
# 示例公司 company_link
test_link = companies["company_link"][0]

# 初始化结果列表
company_data = []

# 打开链接
driver.get(test_link)
time.sleep(3)  # 等待页面加载

# 使用 BeautifulSoup 解析 HTML
soup = BeautifulSoup(driver.page_source, "html.parser")

# 提取基本信息
company_info = extract_table_data("jbzl_table")
issuance_info = extract_table_data("fxxg_table")

In [5]:
company_info

{'公司名称': '天齐锂业股份有限公司',
 '英文名称': 'Tianqi Lithium Corporation',
 'A股代码': '002466',
 'A股简称': '天齐锂业',
 'A股扩位简称': '--',
 '曾用名': '--',
 'B股代码': '--',
 'B股简称': '--',
 'H股代码': '09696.HK',
 'H股简称': '天齐锂业',
 '证券类别': '深交所主板A股',
 '所属东财行业': '有色金属-稀有金属-锂',
 '上市交易所': '深圳证券交易所',
 '所属证监会行业': '制造业-有色金属冶炼和压延加工业',
 '总经理': '夏浚诚',
 '法人代表': '蒋安琪',
 '董秘': '张文宇',
 '董事长': '蒋安琪',
 '证券事务代表': '付旭梅',
 '独立董事': '黄玮(Huang Wei),唐国琼,吴昌华,向川',
 '联系电话': '028-85183501',
 '电子信箱': 'ir@tianqilithium.com',
 '传真': '028-85159451',
 '公司网址': 'www.tianqilithium.com',
 '办公地址': '四川省成都市天府新区红梁西一街166号',
 '注册地址': '四川省射洪县太和镇城北',
 '区域': '四川',
 '邮政编码': '610299',
 '注册资本(元)': '16.41亿',
 '工商登记': '91510900206360802D',
 '雇员人数': '2864',
 '管理人员人数': '17',
 '律师事务所': '北京中伦(成都)律师事务所',
 '会计师事务所': '毕马威华振会计师事务所(特殊普通合伙),毕马威会计师事务所',
 '公司简介': '天齐锂业股份有限公司是以锂为核心的新能源材料企业,为深圳证券交易所(SZ.002466)和香港联合交易所(9696.HK)两地上市公司,公司业务涵盖锂产业链的关键阶段,包括硬岩型锂矿资源的开发、锂精矿加工销售以及锂化工产品的生产销售。公司战略性布局中国、澳大利亚和智利的锂资源,并凭借垂直一体化的全球产业链优势与国际客户建立伙伴关系,共同助力电动汽车和储能产业实现锂离子电池技术的长期可持续发展。过去的30余年间,天齐锂业经历了企业改制

In [7]:
issuance_info

{'保荐机构': '国金证券股份有限公司',
 '主承销商': '国金证券股份有限公司',
 '成立日期': '1995-10-16',
 '上市日期': '2010-08-31',
 '发行市盈率(倍)': '90.36',
 '网上发行日期': '2010-08-18',
 '发行方式': '网下询价配售',
 '每股面值(元)': '1',
 '发行量(股)': '2450万',
 '每股发行价(元)': '30.00',
 '发行费用(元)': '5743万',
 '发行总市值(元)': '7.350亿',
 '募集资金净额(元)': '6.776亿',
 '首日开盘价(元)': '76.00',
 '首日收盘价(元)': '83.05',
 '首日换手率': '86.31%',
 '首日最高价(元)': '85.16',
 '网下配售中签率': '0.60%',
 '定价中签率': '0.40%'}

In [10]:
save_to_word("天齐锂业", company_info, issuance_info)

Word 文件已保存: company_info.docx
